Copyright 2019 The TensorFlow Authors.

```
@title Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License.
You may obtain a copy of the License at
https://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
```

# Dịch máy thần kinh với sự chú ý

Sổ tay này huấn luyện một mô hình chuỗi đến chuỗi mô hình (seq2seq) cho ngôn ngữ Tây Ban Nha sang bản dịch tiếng Anh dựa trên hiệu quả [Effective Approaches to Attention-based Neural Machine Translation](https://arxiv.org/abs/1508.04025v5) . Đây là một ví dụ nâng cao giả định một số kiến ​​thức về:

* Các mô hình seq2seq

* Các nguyên tắc cơ bản của TensorFlow bên dưới lớp keras:

 * Làm việc trực tiếp với tensors

 * Viết tùy chỉnh các `keras.Model` và `keras.layers`

Trong khi kiến trúc này có phần lỗi thời, nó vẫn là một dự án rất hữu ích cho công việc thông qua để có được một sự hiểu biết sâu sắc hơn về cơ chế chú ý (trước khi đi vào [Transformers](https://render.githubusercontent.com/view/transformer.ipynb)).

Sau khi đào tạo mô hình trong máy tính xách tay này, bạn sẽ có thể nhập vào một câu tiếng Tây Ban Nha, chẳng hạn như "*¿todavia estan en casa?*", và gửi lại bản dịch tiếng Anh: "*are you still at home?*"

Mô hình kết quả có thể xuất như là một mô hình `tf.saved_model`, vì vậy nó có thể được sử dụng trong các môi trường TensorFlow khác.

Chất lượng bản dịch là hợp lý đối với một ví dụ về đồ chơi, nhưng đồ thì sự chú ý được tạo ra có lẽ thú vị hơn. Điều này cho thấy phần nào của câu đầu vào được mô hình chú ý trong khi dịch:

![spanish-english](img/spanish-english.png)

Lưu ý: Ví dụ này mất khoảng 10 phút để chạy trên một GPU P100 duy nhất.

## Cài đặt

In [ ]:
!pip install tensorflow_text

In [ ]:
import numpy as np

import typing
from typing import Any, Tuple

import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import tensorflow_text as tf_text

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

Hướng dẫn này xây dựng một vài lớp từ đầu, hãy sử dụng biến này nếu bạn muốn chuyển đổi giữa triển khai tùy chỉnh và tích hợp.

In [ ]:
use_builtins = True

Hướng dẫn này sử dụng rất nhiều API cấp thấp, nơi rất dễ làm sai hình dạng. Lớp này được sử dụng để kiểm tra các hình dạng trong suốt hướng dẫn.


In [ ]:
#@title Kiểm tra hình dạng
class ShapeChecker():
  def __init__(self):
    # Keep a cache of every axis-name seen
    self.shapes = {}

  def __call__(self, tensor, names, broadcast=False):
    if not tf.executing_eagerly():
      return

    if isinstance(names, str):
      names = (names,)

    shape = tf.shape(tensor)
    rank = tf.rank(tensor)

    if rank != len(names):
      raise ValueError(f'Rank mismatch:\n'
                       f'    found {rank}: {shape.numpy()}\n'
                       f'    expected {len(names)}: {names}\n')

    for i, name in enumerate(names):
      if isinstance(name, int):
        old_dim = name
      else:
        old_dim = self.shapes.get(name, None)
      new_dim = shape[i]

      if (broadcast and new_dim == 1):
        continue

      if old_dim is None:
        # If the axis name is new, add its length to the cache.
        self.shapes[name] = new_dim
        continue

      if new_dim != old_dim:
        raise ValueError(f"Shape mismatch for dimension: '{name}'\n"
                         f"    found: {new_dim}\n"
                         f"    expected: {old_dim}\n")

## Dữ liệu

Chúng tôi sẽ sử dụng một tập dữ liệu ngôn ngữ được cung cấp bởi http://www.manythings.org/anki/. Bộ dữ liệu này chứa các cặp dịch ngôn ngữ trong định dạng:

```
May I borrow this book?	¿Puedo tomar prestado este libro?
```

Họ có nhiều ngôn ngữ khác nhau, nhưng chúng tôi sẽ sử dụng tập dữ liệu tiếng Anh-Tây Ban Nha.

### Tải xuống và chuẩn bị tập dữ liệu

Để thuận tiện, chúng tôi đã lưu trữ bản sao của tập dữ liệu này trên Google Cloud, nhưng bạn cũng có thể tải xuống bản sao của riêng mình. Sau khi tải xuống tập dữ liệu, đây là các bước chúng tôi sẽ thực hiện để chuẩn bị dữ liệu:

1. Thêm một token *start* và *end* cho mỗi câu.

2. Làm sạch các câu bằng cách loại bỏ các ký tự đặc biệt.

3. Tạo chỉ mục từ và chỉ mục từ đảo ngược (ánh xạ từ điển từ word → id và id → word).

4. Đệm mỗi câu đến độ dài tối đa.

In [ ]:
# Download the file
import pathlib

path_to_zip = tf.keras.utils.get_file(
    'spa-eng.zip', origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
    extract=True)

path_to_file = pathlib.Path(path_to_zip).parent/'spa-eng/spa.txt'

In [ ]:
def load_data(path):
  text = path.read_text(encoding='utf-8')

  lines = text.splitlines()
  pairs = [line.split('\t') for line in lines]

  inp = [inp for targ, inp in pairs]
  targ = [targ for targ, inp in pairs]

  return targ, inp

In [ ]:
targ, inp = load_data(path_to_file)
print(inp[-1])

In [ ]:
print(targ[-1])

### Tạo tập dữ liệu tf.data

Từ những mảng của chuỗi bạn có thể tạo một `tf.data.Dataset` các chuỗi xáo trộn và cho vào lô một cách hiệu quả:

In [ ]:
BUFFER_SIZE = len(inp)
BATCH_SIZE = 64

dataset = tf.data.Dataset.from_tensor_slices((inp, targ)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)

In [ ]:
for example_input_batch, example_target_batch in dataset.take(1):
  print(example_input_batch[:5])
  print()
  print(example_target_batch[:5])
  break

### Tiền xử lý văn bản

Một trong những mục tiêu của hướng dẫn này là xây dựng một mô hình có thể được xuất ra dưới dạng một `tf.saved_model`. Để thực hiện mô hình đã xuất hữu ích, nó nên lấy các đầu vào `tf.string`, và trả về các đầu ra `tf.string`: Tất cả quá trình xử lý văn bản diễn ra bên trong mô hình.

#### Chuẩn hoá

Mô hình đang xử lý văn bản đa ngôn ngữ với vốn từ vựng hạn chế. Vì vậy, việc chuẩn hóa văn bản đầu vào sẽ rất quan trọng.

Bước đầu tiên là chuẩn hóa Unicode để tách các ký tự có dấu và thay thế các ký tự tương thích bằng các ký tự tương đương ASCII của chúng.

Các `tensroflow_text` gói chứa một hoạt động bình thường hóa unicode:

In [ ]:
example_text = tf.constant('¿Todavía está en casa?')

print(example_text.numpy())
print(tf_text.normalize_utf8(example_text, 'NFKD').numpy())

Chuẩn hóa Unicode sẽ là bước đầu tiên trong hàm chuẩn hóa văn bản:

In [ ]:
def tf_lower_and_split_punct(text):
  # Split accecented characters.
  text = tf_text.normalize_utf8(text, 'NFKD')
  text = tf.strings.lower(text)
  # Keep space, a to z, and select punctuation.
  text = tf.strings.regex_replace(text, '[^ a-z.?!,¿]', '')
  # Add spaces around punctuation.
  text = tf.strings.regex_replace(text, '[.?!,¿]', r' \0 ')
  # Strip whitespace.
  text = tf.strings.strip(text)

  text = tf.strings.join(['[START]', text, '[END]'], separator=' ')
  return text

In [ ]:
print(example_text.numpy().decode())
print(tf_lower_and_split_punct(example_text).numpy().decode())

#### Véctơ hoá văn bản

Chức năng chuẩn hoá này sẽ được gói trong một lớp `preprocessing.TextVectorization` lớp mà sẽ xử lý việc khai thác vốn từ vựng và chuyển đổi văn bản đầu vào thành các chuối token.

In [ ]:
max_vocab_size = 5000

input_text_processor = preprocessing.TextVectorization(
    standardize=tf_lower_and_split_punct,
    max_tokens=max_vocab_size)

Các lớp `TextVectorization` và các lớp `experimental.preprocessing` khác có một phương thức `adapt`. Phương thức này lần đọc một epoch của dữ liệu huấn luyện, và hoạt động rất nhiều giống `Model.fix`. Phương thức `adapt` này khởi tạo các lớp dựa trên dữ liệu. Ở đây nó xác định từ vựng:

In [ ]:
input_text_processor.adapt(inp)

# Here are the first 10 words from the vocabulary:
input_text_processor.get_vocabulary()[:10]

Đó là lớp `TextVectorization` tiếng Tây Ban Nha, bây giờ xây dựng và `.adapt()` tiếng Anh cái này:

In [ ]:
output_text_processor = preprocessing.TextVectorization(
    standardize=tf_lower_and_split_punct,
    max_tokens=max_vocab_size)

output_text_processor.adapt(targ)
output_text_processor.get_vocabulary()[:10]

Giờ đây, các lớp này có thể chuyển đổi một lô các chuỗi thành một lô các token ID:

In [ ]:
example_tokens = input_text_processor(example_input_batch)
example_tokens[:3, :10]

Phương thức `get_vocabulary` có thể được sử dụng để chuyển đổi các token ID trở lại thành văn bản:

In [ ]:
input_vocab = np.array(input_text_processor.get_vocabulary())
tokens = input_vocab[example_tokens[0].numpy()]
' '.join(tokens)

Các token ID được trả về không có đệm. Điều này có thể dễ dàng để trở thành mặt nạ:

In [ ]:
plt.subplot(1, 2, 1)
plt.pcolormesh(example_tokens)
plt.title('Token IDs')

plt.subplot(1, 2, 2)
plt.pcolormesh(example_tokens != 0)
plt.title('Mask')

## Mô hình bộ mã hoá/giải mã

Sơ đồ sau đây cho thấy tổng quan về mô hình. Tại mỗi bước thời gian, đầu ra của bộ giải mã được kết hợp với tổng trọng số trên đầu vào được mã hóa, để dự đoán từ tiếp theo. Sơ đồ và công thức từ [bài báo của Lương](https://arxiv.org/abs/1508.04025v5).

![attention_mechanism](img/attention_mechanism.jpg)


Trước khi đi sâu vào nó, hãy xác định một vài hằng số cho mô hình:

In [ ]:
embedding_dim = 256
units = 1024

### Bộ mã hoá

Bắt đầu bằng cách xây dựng bộ mã hóa, phần màu xanh lam của sơ đồ ở trên.

Bộ mã hóa:

1. Lấy một danh sách các token ID (từ `input_text_processor`).

2. Tra cứu một véctơ nhúng cho mỗi token (Sử dụng một `layers.Embedding`).

3. Xử lý các nhúng thành một chuỗi mới (Sử dụng một `layers.GRU`).

4. Trả về:
    * Chuỗi đã xử lý. Điều này sẽ được chuyển đến phía chú ý.
    * Trạng thái bên trong. Điều này sẽ được sử dụng để khởi tạo bộ giải mã

In [ ]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, input_vocab_size, embedding_dim, enc_units):
    super(Encoder, self).__init__()
    self.enc_units = enc_units
    self.input_vocab_size = input_vocab_size

    # The embedding layer converts tokens to vectors
    self.embedding = tf.keras.layers.Embedding(self.input_vocab_size,
                                               embedding_dim)

    # The GRU RNN layer processes those vectors sequentially.
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   # Return the sequence and state
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, tokens, state=None):
    shape_checker = ShapeChecker()
    shape_checker(tokens, ('batch', 's'))

    # 2. The embedding layer looks up the embedding for each token.
    vectors = self.embedding(tokens)
    shape_checker(vectors, ('batch', 's', 'embed_dim'))

    # 3. The GRU processes the embedding sequence.
    #    output shape: (batch, s, enc_units)
    #    state shape: (batch, enc_units)
    output, state = self.gru(vectors, initial_state=state)
    shape_checker(output, ('batch', 's', 'enc_units'))
    shape_checker(state, ('batch', 'enc_units'))

    # 4. Returns the new sequence and its state.
    return output, state

Đây là cách nó phù hợp với nhau cho đến nay:

In [ ]:
# Convert the input text to tokens.
example_tokens = input_text_processor(example_input_batch)

# Encode the input sequence.
encoder = Encoder(input_text_processor.vocabulary_size(),
                  embedding_dim, units)
example_enc_output, example_enc_state = encoder(example_tokens)

print(f'Input batch, shape (batch): {example_input_batch.shape}')
print(f'Input batch tokens, shape (batch, s): {example_tokens.shape}')
print(f'Encoder output, shape (batch, s, units): {example_enc_output.shape}')
print(f'Encoder state, shape (batch, units): {example_enc_state.shape}')

Bộ mã hóa trả về trạng thái bên trong của nó để trạng thái của nó có thể được sử dụng để khởi tạo bộ giải mã.

RNN cũng thường trả về trạng thái của nó để nó có thể xử lý một chuỗi qua nhiều cuộc gọi. Bạn sẽ thấy nhiều hơn về việc xây dựng bộ giải mã.

### Phía chú ý

Bộ giải mã sử dụng sự chú ý để tập trung có chọn lọc vào các phần của chuỗi đầu vào. Sự chú ý lấy một chuỗi các véctơ làm đầu vào cho mỗi ví dụ và trả về một véctơ "chú ý" cho mỗi ví dụ. Lớp chú ý này cũng tương tự như một `layers.GlobalAveragePoling1D` nhưng lớp chú ý thực hiện một trung bình *có trọng số*.

Hãy xem cách này hoạt động như thế nào:

1. Phương trình đầu tiên:

![equation_1](img/attention_equation_1.jpg)

```
$$ \alpha_{ts}=\frac{\mathrm{exp}\left (\mathrm{score} \left (\boldsymbol{h}_t,\boldsymbol{\overline{h}}_s \right )\right )}{\sum_{s'=1}^{S}\mathrm{exp}\left (\mathrm{score\left (\boldsymbol{h}_t,\boldsymbol{\overline{h}}_{s'} \right )} \right )} \qquad \textrm{[Attention weights]}\qquad(1) $$
```

2. Phương trình thứ hai:

![equation_2](img/attention_equation_2.jpg)

```
\boldsymbol{c}_t = \sum_{s}^{}\alpha_{ts}\boldsymbol{\overline{h}}_s \qquad \textrm{[Context vector]} \qquad(2)
```

Trong đó:

* $s$ là chỉ số bộ mã hoá.
* $t$ là chỉ số bộ giải mã.
* $\alpha_{ts}$ là các trọng số chú ý.
* $h_s$ là chuỗi của đầu ra bộ mã hoá được tham gia (sự chú ý "key" và "value" trong thuật ngữ transformer).
* $h_t$ là trạng thái bộ giải mã tham gia vào chuỗi (sự chú ý "query" có trong thuật ngữ transformer).
* $c_t$ là véctơ ngữ cảnh kết quả.
* $a_t$ là đầu ra cuối cùng kết hợp "context" và "query".

Các phương trình:

1. Tính toán các trọng số chú ý, $\alpha_{ts}$, như một softmax ngang qua chuỗi đầu ra của bộ mã hoá.
2. Tính toán véctơ ngữ cảnh như tổng các đầu ra bộ mã hoá có trọng số.


Cuối cùng là hàm $\textrm{score}$. Công việc của nó là tính toán điểm-logit vô hướng cho mỗi cặp khóa-truy vấn. Có hai cách tiếp cận phổ biến:

$$
\mathrm{score}\left (\boldsymbol{h}_t,\boldsymbol{\overline{h}}_s\right )=\left\{\begin{matrix}
\boldsymbol{{h}^{\top}_{t}W\overline{h}_s}\qquad\textrm{[Luong's multiplicative style]}\\ 
\boldsymbol{\upsilon }^{\top}_a\mathrm{tanh}\left (\boldsymbol{W_1h_t}+\boldsymbol{W_2\overline{h}_s} \right )\qquad\textrm{[Bahdanau's additive style]}
\end{matrix}\right. (4)
$$

Hướng dẫn này sử dụng [Bahdanau's additive attention](https://arxiv.org/pdf/1409.0473.pdf). TensorFlow bao gồm các triển khai của cả hai `layers.Attention` và
`layers.AdditiveAttention`. Lớp bên dưới xử lý các ma trận trọng số trong một cặp của các lớp `layers.Dense`, và gọi triển khai tích hợp.

In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super().__init__()
    # For Eqn. (4), the  Bahdanau attention
    self.W1 = tf.keras.layers.Dense(units, use_bias=False)
    self.W2 = tf.keras.layers.Dense(units, use_bias=False)

    self.attention = tf.keras.layers.AdditiveAttention()

  def call(self, query, value, mask):
    shape_checker = ShapeChecker()
    shape_checker(query, ('batch', 't', 'query_units'))
    shape_checker(value, ('batch', 's', 'value_units'))
    shape_checker(mask, ('batch', 's'))

    # From Eqn. (4), `W1@ht`.
    w1_query = self.W1(query)
    shape_checker(w1_query, ('batch', 't', 'attn_units'))

    # From Eqn. (4), `W2@hs`.
    w2_key = self.W2(value)
    shape_checker(w2_key, ('batch', 's', 'attn_units'))

    query_mask = tf.ones(tf.shape(query)[:-1], dtype=bool)
    value_mask = mask

    context_vector, attention_weights = self.attention(
        inputs = [w1_query, value, w2_key],
        mask=[query_mask, value_mask],
        return_attention_scores = True,
    )
    shape_checker(context_vector, ('batch', 't', 'value_units'))
    shape_checker(attention_weights, ('batch', 't', 's'))

    return context_vector, attention_weights

### Kiểm tra lớp Chú ý

Tạo một lớp `BahdanauAttention`:

In [ ]:
attention_layer = BahdanauAttention(units)

Lớp này có 3 đầu vào:

* `query`: Nó sẽ được tạo ra bởi bộ giải mã sau này..
* `value`: Nó sẽ là đầu ra của bộ mã hoá.
* `mask`: Ngoại trừ đệm, `example_tokens != 0`

In [ ]:
(example_tokens != 0).shape

Việc triển khai véctơ hóa của lớp chú ý cho phép bạn chuyển một lô chuỗi các vectơ truy vấn và một lô chuỗi các vectơ giá trị. Kết quả là:

1. Một lô của các chuỗi của các véctơ kết quả với kích thước của các truy vấn.

2. Một lô các bản đồ chú ý, với kích thước `(query_length, value_length)`.

In [ ]:
# Later, the decoder will generate this attention query
example_attention_query = tf.random.normal(shape=[len(example_tokens), 2, 10])

# Attend to the encoded tokens

context_vector, attention_weights = attention_layer(
    query=example_attention_query,
    value=example_enc_output,
    mask=(example_tokens != 0))

print(f'Attention result shape: (batch_size, query_seq_length, units):           {context_vector.shape}')
print(f'Attention weights shape: (batch_size, query_seq_length, value_seq_length): {attention_weights.shape}')

Các trọng số chú ý nên tổng hợp thành 1.0 cho mỗi chuỗi.

Dưới đây là các trọng số chú ý trên chuỗi tại `t=0`:

In [ ]:
plt.subplot(1, 2, 1)
plt.pcolormesh(attention_weights[:, 0, :])
plt.title('Attention weights')

plt.subplot(1, 2, 2)
plt.pcolormesh(example_tokens != 0)
plt.title('Mask')


Bởi vì sự khởi tạo ngẫu nhiên nhỏ các trọng số chú ý tất cả gần `1/(sequence_length)`. Nếu bạn phóng to trên các trọng số cho một chuỗi duy nhất, bạn có thể thấy rằng có một số sự thay đổi *nhỏ* rằng mô hình có thể học hỏi để mở rộng và khai thác.

In [ ]:
attention_weights.shape

In [ ]:
attention_slice = attention_weights[0, 0].numpy()
attention_slice = attention_slice[attention_slice != 0]

In [ ]:
#@title
plt.suptitle('Attention weights for one sequence')

plt.figure(figsize=(12, 6))
a1 = plt.subplot(1, 2, 1)
plt.bar(range(len(attention_slice)), attention_slice)
# freeze the xlim
plt.xlim(plt.xlim())
plt.xlabel('Attention weights')

a2 = plt.subplot(1, 2, 2)
plt.bar(range(len(attention_slice)), attention_slice)
plt.xlabel('Attention weights, zoomed')

# zoom in
top = max(a1.get_ylim())
zoom = 0.85*top
a2.set_ylim([0.90*top, top])
a1.plot(a1.get_xlim(), [zoom, zoom], color='k')

### Bộ giải mã

Công việc của bộ giải mã là tạo ra các dự đoán cho token đầu ra tiếp theo.

1. Bộ giải mã nhận được đầu ra bộ mã hóa hoàn chỉnh.

2. Nó sử dụng RNN để theo dõi những gì nó đã tạo ra cho đến nay.

3. Nó sử dụng đầu ra RNN làm truy vấn để truy vấn sự chú ý qua đầu ra của bộ mã hóa, tạo ra vectơ ngữ cảnh.

4. Nó kết hợp đầu ra RNN và vectơ ngữ cảnh bằng cách sử dụng Công thức 3 (bên dưới) để tạo ra "vectơ chú ý".

5. Nó tạo ra các dự đoán logit cho token tiếp theo dựa trên "vectơ chú ý".

$$
\boldsymbol{\alpha}_t=f(\boldsymbol{c}_t,\boldsymbol{h}=\mathrm{tanh}\left (\boldsymbol{W_c[c_t;h_t]} \right )\qquad\textrm{[Attention vector]}
$$

Đây là lớp `Decoder` và khởi tạo của nó. Trình khởi tạo tạo tất cả các lớp cần thiết.

In [ ]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, output_vocab_size, embedding_dim, dec_units):
    super(Decoder, self).__init__()
    self.dec_units = dec_units
    self.output_vocab_size = output_vocab_size
    self.embedding_dim = embedding_dim

    # For Step 1. The embedding layer convets token IDs to vectors
    self.embedding = tf.keras.layers.Embedding(self.output_vocab_size,
                                               embedding_dim)

    # For Step 2. The RNN keeps track of what's been generated so far.
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

    # For step 3. The RNN output will be the query for the attention layer.
    self.attention = BahdanauAttention(self.dec_units)

    # For step 4. Eqn. (3): converting `ct` to `at`
    self.Wc = tf.keras.layers.Dense(dec_units, activation=tf.math.tanh,
                                    use_bias=False)

    # For step 5. This fully connected layer produces the logits for each
    # output token.
    self.fc = tf.keras.layers.Dense(self.output_vocab_size)

Phương thức `call` cho lớp này lấy và trả về nhiều tensors. Tổ chức chúng thành các lớp chứa đơn giản:

In [ ]:
class DecoderInput(typing.NamedTuple):
  new_tokens: Any
  enc_output: Any
  mask: Any

class DecoderOutput(typing.NamedTuple):
  logits: Any
  attention_weights: Any

Đây là việc triển khai phương thức `call`:

In [ ]:
def call(self,
         inputs: DecoderInput,
         state=None) -> Tuple[DecoderOutput, tf.Tensor]:
  shape_checker = ShapeChecker()
  shape_checker(inputs.new_tokens, ('batch', 't'))
  shape_checker(inputs.enc_output, ('batch', 's', 'enc_units'))
  shape_checker(inputs.mask, ('batch', 's'))

  if state is not None:
    shape_checker(state, ('batch', 'dec_units'))

  # Step 1. Lookup the embeddings
  vectors = self.embedding(inputs.new_tokens)
  shape_checker(vectors, ('batch', 't', 'embedding_dim'))

  # Step 2. Process one step with the RNN
  rnn_output, state = self.gru(vectors, initial_state=state)

  shape_checker(rnn_output, ('batch', 't', 'dec_units'))
  shape_checker(state, ('batch', 'dec_units'))

  # Step 3. Use the RNN output as the query for the attention over the
  # encoder output.
  context_vector, attention_weights = self.attention(
      query=rnn_output, value=inputs.enc_output, mask=inputs.mask)
  shape_checker(context_vector, ('batch', 't', 'dec_units'))
  shape_checker(attention_weights, ('batch', 't', 's'))

  # Step 4. Eqn. (3): Join the context_vector and rnn_output
  #     [ct; ht] shape: (batch t, value_units + query_units)
  context_and_rnn_output = tf.concat([context_vector, rnn_output], axis=-1)

  # Step 4. Eqn. (3): `at = tanh(Wc@[ct; ht])`
  attention_vector = self.Wc(context_and_rnn_output)
  shape_checker(attention_vector, ('batch', 't', 'dec_units'))

  # Step 5. Generate logit predictions:
  logits = self.fc(attention_vector)
  shape_checker(logits, ('batch', 't', 'output_vocab_size'))

  return DecoderOutput(logits, attention_weights), state

In [ ]:
Decoder.call = call

Các **bộ mã hóa** xử lý chuỗi đầu vào đầy đủ của nó với một cuộc gọi duy nhất đến RNN của nó. Thực hiện này của các **bộ giải mã** *có thể* làm điều đó cũng cho hiệu quả huấn luyện. Nhưng hướng dẫn này sẽ chạy bộ giải mã trong một vòng lặp vì một số lý do:

* Linh hoạt: Viết vòng lặp cho phép bạn kiểm soát trực tiếp quy trình huấn luyện.
* Rõ ràng: Có thể làm mặt nạ thủ thuật và sử dụng `layers.RNN`, hoặc API `tfa.seq2seq` để đóng gói tất cả thành một cuộc gọi duy nhất. Nhưng viết nó ra dưới dạng một vòng lặp có thể rõ ràng hơn.

    * Vòng lặp huấn luyện miễn phí được trình bày trong hướng dẫn [sinh văn bản](https://www.tensorflow.org/text/tutorials/text_generation).


Bây giờ hãy thử sử dụng bộ giải mã này.

In [ ]:
decoder = Decoder(output_text_processor.vocabulary_size(),
                  embedding_dim, units)

Bộ giải mã có 4 đầu vào.

* `new_tokens` - Các token cuối cùng được tạo. Khởi tạo các bộ giải mã với token `"[START]"`.

* `enc_output` - được tạo ra bởi `Encoder`.

* `mask` - Một tensor boolean chỉ ra nơi `tokens != 0`

* `state` - Các đầu ra `state` trước từ các bộ giải mã (trạng thái nội bộ của RNN của bộ giải mã). Vượt qua `None` để khởi tạo-không cho nó. Bài gốc khởi tạo nó từ trạng thái RNN cuối cùng của bộ mã hóa.

In [ ]:
# Convert the target sequence, and collect the "[START]" tokens
example_output_tokens = output_text_processor(example_target_batch)

start_index = output_text_processor._index_lookup_layer('[START]').numpy()
first_token = tf.constant([[start_index]] * example_output_tokens.shape[0])

In [ ]:
# Run the decoder
dec_result, dec_state = decoder(
    inputs = DecoderInput(new_tokens=first_token,
                          enc_output=example_enc_output,
                          mask=(example_tokens != 0)),
    state = example_enc_state
)

print(f'logits shape: (batch_size, t, output_vocab_size) {dec_result.logits.shape}')
print(f'state shape: (batch_size, dec_units) {dec_state.shape}')

Lấy mẫu token theo các logit:

In [ ]:
sampled_token = tf.random.categorical(dec_result.logits[:, 0, :], num_samples=1)

Giải mã token dưới dạng từ đầu tiên của đầu ra:

In [ ]:
vocab = np.array(output_text_processor.get_vocabulary())
first_word = vocab[sampled_token.numpy()]
first_word[:5]

Bây giờ, hãy sử dụng bộ giải mã để tạo bộ logit thứ hai.

* Chuyển cùng `enc_output` và `mask`, những điều này không thay đổi.

* Chuyển qua mẫu token như `new_tokens`.

* Chuyển `decoder_state` bộ giải mã trả về lần cuối cùng, vì vậy RNN tiếp tục với một bộ nhớ về nơi nó rời khỏi thời gian qua.

In [ ]:
dec_result, dec_state = decoder(
    DecoderInput(sampled_token,
                 example_enc_output,
                 mask=(example_tokens != 0)),
    state=dec_state)

In [ ]:
sampled_token = tf.random.categorical(dec_result.logits[:, 0, :], num_samples=1)
first_word = vocab[sampled_token.numpy()]
first_word[:5]

## Huấn luyện

Bây giờ bạn đã có tất cả các thành phần của mô hình, đã đến lúc bắt đầu huấn luyện mô hình. Có thể bạn sẽ cần:

* Một hàm mất mát và trình tối ưu hóa để thực hiện việc tối ưu hóa.

* Hàm bước huấn luyện xác định cách cập nhật mô hình cho từng lô đầu vào/mục tiêu.

* Một vòng lặp huấn luyện để thúc đẩy quá trình huấn luyện và lưu các điểm kiểm tra.

### Định nghĩa hàm mất mát

In [ ]:
class MaskedLoss(tf.keras.losses.Loss):
  def __init__(self):
    self.name = 'masked_loss'
    self.loss = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none')

  def __call__(self, y_true, y_pred):
    shape_checker = ShapeChecker()
    shape_checker(y_true, ('batch', 't'))
    shape_checker(y_pred, ('batch', 't', 'logits'))

    # Calculate the loss for each item in the batch.
    loss = self.loss(y_true, y_pred)
    shape_checker(loss, ('batch', 't'))

    # Mask off the losses on padding.
    mask = tf.cast(y_true != 0, tf.float32)
    shape_checker(mask, ('batch', 't'))
    loss *= mask

    # Return the total.
    return tf.reduce_sum(loss)

### Triển khai bước huấn luyện

Bắt đầu với một lớp mô hình, quá trình huấn luyện sẽ được triển khai như phương thức `train_step` trên mô hình này. Xem [Tuỳ chỉnh phù hợp](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit) để biết chi tiết.

Ở đây phương thức `train_step` là một gói bọc quanh triển khai `_train_step` sẽ đến sau. Gói bao bọc này bao gồm một công tắc để bật và tắt biên dịch `tf.function`, để cho gỡ lỗi dễ dàng hơn.

In [ ]:
class TrainTranslator(tf.keras.Model):
  def __init__(self, embedding_dim, units,
               input_text_processor,
               output_text_processor, 
               use_tf_function=True):
    super().__init__()
    # Build the encoder and decoder
    encoder = Encoder(input_text_processor.vocabulary_size(),
                      embedding_dim, units)
    decoder = Decoder(output_text_processor.vocabulary_size(),
                      embedding_dim, units)

    self.encoder = encoder
    self.decoder = decoder
    self.input_text_processor = input_text_processor
    self.output_text_processor = output_text_processor
    self.use_tf_function = use_tf_function
    self.shape_checker = ShapeChecker()

  def train_step(self, inputs):
    self.shape_checker = ShapeChecker()
    if self.use_tf_function:
      return self._tf_train_step(inputs)
    else:
      return self._train_step(inputs)

Nhìn chung triển khai cho phương thức `Model.train_step` là như sau:

1. Nhận một lô `input_text`, `target_text` từ `tf.data.Dataset`.

2. Chuyển đổi các đầu vào văn bản thô đó thành token-nhúng và mặt nạ.

3. Chạy bộ mã hóa trên `input_tokens` để có được `encoder_output` và `encoder_state`.
4. Khởi tạo trạng thái bộ giải mã và mất mát.

5. Vòng lặp qua `target_tokens`:

    a. Chạy bộ giải mã từng bước một.

    b. Tính toán sự mất mát cho mỗi bước.

    c. Tích lũy mất mát trung bình.

6. Tính độ dốc của mất mát và sử dụng tối ưu hóa để áp dụng bản cập nhật trên trainable_variables` của mô hình.

Phương thức `_preprocess` được bổ sung dưới đây, thực hiện bước #1 và #2:

In [ ]:
def _preprocess(self, input_text, target_text):
  self.shape_checker(input_text, ('batch',))
  self.shape_checker(target_text, ('batch',))

  # Convert the text to token IDs
  input_tokens = self.input_text_processor(input_text)
  target_tokens = self.output_text_processor(target_text)
  self.shape_checker(input_tokens, ('batch', 's'))
  self.shape_checker(target_tokens, ('batch', 't'))

  # Convert IDs to masks.
  input_mask = input_tokens != 0
  self.shape_checker(input_mask, ('batch', 's'))

  target_mask = target_tokens != 0
  self.shape_checker(target_mask, ('batch', 't'))

  return input_tokens, input_mask, target_tokens, target_mask

In [ ]:
TrainTranslator._preprocess = _preprocess

Phương thức `_train_step` được thêm vào bên dưới, xử lý các bước còn lại ngoại trừ thực sự chạy bộ giải mã:

In [ ]:
def _train_step(self, inputs):
  input_text, target_text = inputs  

  (input_tokens, input_mask,
   target_tokens, target_mask) = self._preprocess(input_text, target_text)

  max_target_length = tf.shape(target_tokens)[1]

  with tf.GradientTape() as tape:
    # Encode the input
    enc_output, enc_state = self.encoder(input_tokens)
    self.shape_checker(enc_output, ('batch', 's', 'enc_units'))
    self.shape_checker(enc_state, ('batch', 'enc_units'))

    # Initialize the decoder's state to the encoder's final state.
    # This only works if the encoder and decoder have the same number of
    # units.
    dec_state = enc_state
    loss = tf.constant(0.0)

    for t in tf.range(max_target_length-1):
      # Pass in two tokens from the target sequence:
      # 1. The current input to the decoder.
      # 2. The target the target for the decoder's next prediction.
      new_tokens = target_tokens[:, t:t+2]
      step_loss, dec_state = self._loop_step(new_tokens, input_mask,
                                             enc_output, dec_state)
      loss = loss + step_loss

    # Average the loss over all non padding tokens.
    average_loss = loss / tf.reduce_sum(tf.cast(target_mask, tf.float32))

  # Apply an optimization step
  variables = self.trainable_variables 
  gradients = tape.gradient(average_loss, variables)
  self.optimizer.apply_gradients(zip(gradients, variables))

  # Return a dict mapping metric names to current value
  return {'batch_loss': average_loss}

In [ ]:
TrainTranslator._train_step = _train_step

Phương thức `_loop_step` được bổ sung dưới đây, thực hiện các bộ giải mã và tính toán sự mất mát gia tăng và trạng thái giải mã mới (`dec_state`).

In [ ]:
def _loop_step(self, new_tokens, input_mask, enc_output, dec_state):
  input_token, target_token = new_tokens[:, 0:1], new_tokens[:, 1:2]

  # Run the decoder one step.
  decoder_input = DecoderInput(new_tokens=input_token,
                               enc_output=enc_output,
                               mask=input_mask)

  dec_result, dec_state = self.decoder(decoder_input, state=dec_state)
  self.shape_checker(dec_result.logits, ('batch', 't1', 'logits'))
  self.shape_checker(dec_result.attention_weights, ('batch', 't1', 's'))
  self.shape_checker(dec_state, ('batch', 'dec_units'))

  # `self.loss` returns the total for non-padded tokens
  y = target_token
  y_pred = dec_result.logits
  step_loss = self.loss(y, y_pred)

  return step_loss, dec_state

In [ ]:
TrainTranslator._loop_step = _loop_step

### Kiểm tra bước huấn luyện

Xây dựng một `TrainTranslator`, và cấu hình nó cho huấn luyện bằng cách sử dụng phương thức `Model.compile`:

In [ ]:
translator = TrainTranslator(
    embedding_dim, units,
    input_text_processor=input_text_processor,
    output_text_processor=output_text_processor,
    use_tf_function=False)

# Configure the loss and optimizer
translator.compile(
    optimizer=tf.optimizers.Adam(),
    loss=MaskedLoss(),
)

Kiểm tra train_step`. Đối với một mô hình văn bản như thế này, sự mất mát sẽ bắt đầu gần:

In [ ]:
np.log(output_text_processor.vocabulary_size())

In [ ]:
%%time
for n in range(10):
  print(translator.train_step([example_input_batch, example_target_batch]))
print()

Trong khi nó dễ dàng hơn để gỡ lỗi mà không có một `tf.function`, nó làm tăng hiệu suất. Vì vậy, bây giờ mà phương thức `_train_step` đang làm việc, hãy thử các `tf.function` -wrapped `_tf_train_step`, để tối đa hóa hiệu suất trong khi huấn luyện:

In [ ]:
@tf.function(input_signature=[[tf.TensorSpec(dtype=tf.string, shape=[None]),
                               tf.TensorSpec(dtype=tf.string, shape=[None])]])
def _tf_train_step(self, inputs):
  return self._train_step(inputs)

In [ ]:
TrainTranslator._tf_train_step = _tf_train_step

In [ ]:
translator.use_tf_function = True

Cuộc gọi đầu tiên sẽ chậm, vì nó theo dõi hàm.

In [ ]:
translator.train_step([example_input_batch, example_target_batch])

Nhưng sau đó nó thường nhanh hơn gấp 2-3 lần so với phương thức `train_step`:

In [ ]:
%%time
for n in range(10):
  print(translator.train_step([example_input_batch, example_target_batch]))
print()

Một bài kiểm tra tốt đối với một mô hình mới là để thấy rằng nó có thể quá khớp một lô đầu vào duy nhất. Hãy thử nó, mất mát sẽ nhanh chóng về 0:

In [ ]:
losses = []
for n in range(100):
  print('.', end='')
  logs = translator.train_step([example_input_batch, example_target_batch])
  losses.append(logs['batch_loss'].numpy())

print()
plt.plot(losses)

Bây giờ bạn đã tự tin rằng bước huấn luyện đang hoạt động, hãy tạo một bản sao mới của mô hình để huấn luyện từ đầu:

In [ ]:
train_translator = TrainTranslator(
    embedding_dim, units,
    input_text_processor=input_text_processor,
    output_text_processor=output_text_processor)

# Configure the loss and optimizer
train_translator.compile(
    optimizer=tf.optimizers.Adam(),
    loss=MaskedLoss(),
)

### Huấn luyện mô hình

Trong khi không có gì sai với viết vòng lặp huấn luyện tùy chỉnh riêng của bạn, thực hiện phương thức `Model.train_step`, như trong phần trước, cho phép bạn chạy `Model.fit` và tránh viết lại tất cả những mã nồi *boiler-plate*. 

Hướng dẫn này chỉ huấn luyện cho một vài epoch, vì vậy sử dụng một `callbacks.Callback` để thu thập lịch sử của mất mát lô cho việc vẽ đồ thị:

In [ ]:
class BatchLogs(tf.keras.callbacks.Callback):
  def __init__(self, key):
    self.key = key
    self.logs = []

  def on_train_batch_end(self, n, logs):
    self.logs.append(logs[self.key])

batch_loss = BatchLogs('batch_loss')

In [ ]:
train_translator.fit(dataset, epochs=3,
                     callbacks=[batch_loss])

In [ ]:
plt.plot(batch_loss.logs)
plt.ylim([0, 3])
plt.xlabel('Batch #')
plt.ylabel('CE/token')

Các bước nhảy có thể nhìn thấy trong đồ thị nằm ở ranh giới epoch.

## Dịch

Bây giờ mà các mô hình được huấn luyện, gọi một hàm để thực hiện dịch đầy đủ `text => text`.

Đối với điều này nhu cầu mô hình để đảo ngược ánh xạ `text => token IDs` được cung cấp bởi các `output_text_processor`. Nó cũng cần biết các ID cho các token đặc biệt. Tất cả điều này được thực hiện trong phương thức khởi tạo cho lớp mới. Việc triển khai phương pháp dịch thực tế sẽ tuân theo.

Nhìn chung, điều này tương tự như vòng lặp huấn luyện, ngoại trừ đầu vào cho bộ giải mã ở mỗi bước thời gian là một mẫu từ dự đoán cuối cùng của bộ giải mã.

In [ ]:
class Translator(tf.Module):

  def __init__(self, encoder, decoder, input_text_processor,
               output_text_processor):
    self.encoder = encoder
    self.decoder = decoder
    self.input_text_processor = input_text_processor
    self.output_text_processor = output_text_processor

    self.output_token_string_from_index = (
        tf.keras.layers.experimental.preprocessing.StringLookup(
            vocabulary=output_text_processor.get_vocabulary(),
            mask_token='',
            invert=True))

    # The output should never generate padding, unknown, or start.
    index_from_string = tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=output_text_processor.get_vocabulary(), mask_token='')
    token_mask_ids = index_from_string(['', '[UNK]', '[START]']).numpy()

    token_mask = np.zeros([index_from_string.vocabulary_size()], dtype=np.bool)
    token_mask[np.array(token_mask_ids)] = True
    self.token_mask = token_mask

    self.start_token = index_from_string('[START]')
    self.end_token = index_from_string('[END]')

In [ ]:
translator = Translator(
    encoder=train_translator.encoder,
    decoder=train_translator.decoder,
    input_text_processor=input_text_processor,
    output_text_processor=output_text_processor,
)

### Chuyển đổi các token ID thành văn bản

Phương thức đầu tiên để thực hiện là `tokens_to_text` mà chuyển đổi từ các token ID thành văn bản có thể đọc được.

In [ ]:
def tokens_to_text(self, result_tokens):
  shape_checker = ShapeChecker()
  shape_checker(result_tokens, ('batch', 't'))
  result_text_tokens = self.output_token_string_from_index(result_tokens)
  shape_checker(result_text_tokens, ('batch', 't'))

  result_text = tf.strings.reduce_join(result_text_tokens,
                                       axis=1, separator=' ')
  shape_checker(result_text, ('batch'))

  result_text = tf.strings.strip(result_text)
  shape_checker(result_text, ('batch',))
  return result_text

In [ ]:
Translator.tokens_to_text = tokens_to_text

Nhập một số token ID ngẫu nhiên và xem những gì nó tạo ra:

In [ ]:
example_output_tokens = tf.random.uniform(
    shape=[5, 2], minval=0, dtype=tf.int64,
    maxval=output_text_processor.vocabulary_size())
translator.tokens_to_text(example_output_tokens).numpy()

### Các mẫu dự đoán từ bộ giải mã

Hàm này nhận các kết quả đầu ra logit của bộ giải mã và lấy mẫu các token ID từ bản phân phối đó:

In [ ]:
def sample(self, logits, temperature):
  shape_checker = ShapeChecker()
  # 't' is usually 1 here.
  shape_checker(logits, ('batch', 't', 'vocab'))
  shape_checker(self.token_mask, ('vocab',))

  token_mask = self.token_mask[tf.newaxis, tf.newaxis, :]
  shape_checker(token_mask, ('batch', 't', 'vocab'), broadcast=True)

  # Set the logits for all masked tokens to -inf, so they are never chosen.
  logits = tf.where(self.token_mask, -np.inf, logits)

  if temperature == 0.0:
    new_tokens = tf.argmax(logits, axis=-1)
  else: 
    logits = tf.squeeze(logits, axis=1)
    new_tokens = tf.random.categorical(logits/temperature,
                                        num_samples=1)
  
  shape_checker(new_tokens, ('batch', 't'))

  return new_tokens

In [ ]:
Translator.sample = sample

Chạy thử chức năng này trên một số đầu vào ngẫu nhiên:

In [ ]:
example_logits = tf.random.normal([5, 1, output_text_processor.vocabulary_size()])
example_output_tokens = translator.sample(example_logits, temperature=1.0)
example_output_tokens

### Triển khai vòng lặp dịch

Đây là một triển khai hoàn chỉnh của vòng lặp dịch văn bản sang văn bản.

Điều này thực hiện thu thập các kết quả vào danh sách python, trước khi sử dụng `tf.concat` tham gia cùng chúng vào các tensor.

Điều này thực hiện tĩnh gỡ cuộn tròn đồ thị lặp `max_length`. Điều này không sao với việc thực thi khát vọng trong python.

In [ ]:
def translate_unrolled(self,
                       input_text, *,
                       max_length=50,
                       return_attention=True,
                       temperature=1.0):
  batch_size = tf.shape(input_text)[0]
  input_tokens = self.input_text_processor(input_text)
  enc_output, enc_state = self.encoder(input_tokens)

  dec_state = enc_state
  new_tokens = tf.fill([batch_size, 1], self.start_token)

  result_tokens = []
  attention = []
  done = tf.zeros([batch_size, 1], dtype=tf.bool)

  for _ in range(max_length):
    dec_input = DecoderInput(new_tokens=new_tokens,
                             enc_output=enc_output,
                             mask=(input_tokens!=0))
    
    dec_result, dec_state = self.decoder(dec_input, state=dec_state)

    attention.append(dec_result.attention_weights)

    new_tokens = self.sample(dec_result.logits, temperature)

    # If a sequence produces an `end_token`, set it `done`
    done = done | (new_tokens == self.end_token)
    # Once a sequence is done it only produces 0-padding.
    new_tokens = tf.where(done, tf.constant(0, dtype=tf.int64), new_tokens)

    # Collect the generated tokens
    result_tokens.append(new_tokens)

    if tf.executing_eagerly() and tf.reduce_all(done):
      break

  # Convert the list of generates token ids to a list of strings.
  result_tokens = tf.concat(result_tokens, axis=-1)
  result_text = self.tokens_to_text(result_tokens)

  if return_attention:
    attention_stack = tf.concat(attention, axis=1)
    return {'text': result_text, 'attention': attention_stack}
  else:
    return {'text': result_text}


In [ ]:
Translator.translate = translate_unrolled

Chạy nó trên một đầu vào đơn giản:

In [ ]:
%%time
input_text = tf.constant([
    'hace mucho frio aqui.', # "It's really cold here."
    'Esta es mi vida.', # "This is my life.""
])

result = translator.translate(
    input_text = input_text)

print(result['text'][0].numpy().decode())
print(result['text'][1].numpy().decode())
print()

Nếu bạn muốn xuất mô hình này, bạn sẽ cần phải gói phương thức này trong một `tf.function`. Việc triển khai cơ bản này có một số vấn đề nếu bạn cố gắng thực hiện điều đó:

1. Các biểu đồ kết quả rất lớn và mất vài giây để tạo, lưu hoặc tải.

2. Bạn không thể phá vỡ từ một vòng tĩnh gỡ cuộn tròn, vì vậy nó sẽ luôn luôn chạy lặp `max_length`, ngay cả khi tất cả các kết quả đầu ra được thực hiện. Nhưng thậm chí sau đó nó nhanh hơn một chút so với việc thực thi khát vọng.


In [ ]:
@tf.function(input_signature=[tf.TensorSpec(dtype=tf.string, shape=[None])])
def tf_translate(self, input_text):
  return self.translate(input_text)

Translator.tf_translate = tf_translate

Chạy `tf.function` một lần để biên dịch nó:

In [ ]:
%%time
result = translator.tf_translate(
    input_text = input_text)

In [ ]:
%%time
result = translator.tf_translate(
    input_text = input_text)

print(result['text'][0].numpy().decode())
print(result['text'][1].numpy().decode())
print()

In [ ]:
#@title [Tuỳ chọn] Sử dụng vòng lặp tượng trưng
def translate_symbolic(self,
                       input_text,
                       *,
                       max_length=50,
                       return_attention=True,
                       temperature=1.0):
  shape_checker = ShapeChecker()
  shape_checker(input_text, ('batch',))

  batch_size = tf.shape(input_text)[0]

  # Encode the input
  input_tokens = self.input_text_processor(input_text)
  shape_checker(input_tokens, ('batch', 's'))

  enc_output, enc_state = self.encoder(input_tokens)
  shape_checker(enc_output, ('batch', 's', 'enc_units'))
  shape_checker(enc_state, ('batch', 'enc_units'))

  # Initialize the decoder
  dec_state = enc_state
  new_tokens = tf.fill([batch_size, 1], self.start_token)
  shape_checker(new_tokens, ('batch', 't1'))

  # Initialize the accumulators
  result_tokens = tf.TensorArray(tf.int64, size=1, dynamic_size=True)
  attention = tf.TensorArray(tf.float32, size=1, dynamic_size=True)
  done = tf.zeros([batch_size, 1], dtype=tf.bool)
  shape_checker(done, ('batch', 't1'))

  for t in tf.range(max_length):
    dec_input = DecoderInput(
        new_tokens=new_tokens, enc_output=enc_output, mask=(input_tokens != 0))

    dec_result, dec_state = self.decoder(dec_input, state=dec_state)

    shape_checker(dec_result.attention_weights, ('batch', 't1', 's'))
    attention = attention.write(t, dec_result.attention_weights)

    new_tokens = self.sample(dec_result.logits, temperature)
    shape_checker(dec_result.logits, ('batch', 't1', 'vocab'))
    shape_checker(new_tokens, ('batch', 't1'))

    # If a sequence produces an `end_token`, set it `done`
    done = done | (new_tokens == self.end_token)
    # Once a sequence is done it only produces 0-padding.
    new_tokens = tf.where(done, tf.constant(0, dtype=tf.int64), new_tokens)

    # Collect the generated tokens
    result_tokens = result_tokens.write(t, new_tokens)

    if tf.reduce_all(done):
      break

  # Convert the list of generated token ids to a list of strings.
  result_tokens = result_tokens.stack()
  shape_checker(result_tokens, ('t', 'batch', 't0'))
  result_tokens = tf.squeeze(result_tokens, -1)
  result_tokens = tf.transpose(result_tokens, [1, 0])
  shape_checker(result_tokens, ('batch', 't'))

  result_text = self.tokens_to_text(result_tokens)
  shape_checker(result_text, ('batch',))

  if return_attention:
    attention_stack = attention.stack()
    shape_checker(attention_stack, ('t', 'batch', 't1', 's'))

    attention_stack = tf.squeeze(attention_stack, 2)
    shape_checker(attention_stack, ('t', 'batch', 's'))

    attention_stack = tf.transpose(attention_stack, [1, 0, 2])
    shape_checker(attention_stack, ('batch', 't', 's'))

    return {'text': result_text, 'attention': attention_stack}
  else:
    return {'text': result_text}

In [ ]:
Translator.translate = translate_symbolic

Việc triển khai ban đầu đã sử dụng danh sách python để thu thập kết quả đầu ra. Điều này sử dụng `tf.range` như vòng lặp, cho phép `tf.autograph` để chuyển đổi các vòng lặp. Sự thay đổi lớn nhất trong việc thực hiện này là việc sử dụng `tf.TensorArray` thay vì python `list` để các tensor tích lũy. `tf.TensorArray` được yêu cầu để thu thập một số biến của tensors trong chế độ đồ thị.

Với việc thực thi khát vọng, bản triển khai này hoạt động ngang bằng với bản gốc:

In [ ]:
%%time
result = translator.translate(
    input_text = input_text)

print(result['text'][0].numpy().decode())
print(result['text'][1].numpy().decode())
print()

Nhưng khi bạn bọc nó trong một `tf.function` bạn sẽ nhận thấy hai sự khác biệt.

In [ ]:
@tf.function(input_signature=[tf.TensorSpec(dtype=tf.string, shape=[None])])
def tf_translate(self, input_text):
  return self.translate(input_text)

Translator.tf_translate = tf_translate

Đầu tiên: Biểu đồ sáng tạo là nhanh hơn nhiều (~ 10 lần), vì nó không tạo các bản sao `max_iterations` của mô hình.

In [ ]:
%%time
result = translator.tf_translate(
    input_text = input_text)

Thứ hai: Hàm đã biên dịch nhanh hơn nhiều trên các đầu vào nhỏ (5 lần trong ví dụ này), vì nó có thể thoát ra khỏi vòng lặp.

In [ ]:
%%time
result = translator.tf_translate(
    input_text = input_text)

print(result['text'][0].numpy().decode())
print(result['text'][1].numpy().decode())
print()

### Trực quan quá trình

Các trọng số chú ý được trả về bởi phương thức `translate` cho thấy nơi mô hình là "đang tìm kiếm" khi nó được tạo ra mỗi token đầu ra.

Vì vậy, tổng sự chú ý trên đầu vào sẽ trả về tất cả những thứ:

In [ ]:
a = result['attention'][0]

print(np.sum(a, axis=-1))

Đây là sự phân bố sự chú ý cho bước đầu ra đầu tiên của ví dụ đầu tiên. Lưu ý rằng sự chú ý bây giờ tập trung hơn nhiều so với mô hình chưa được huấn luyện:

In [ ]:
_ = plt.bar(range(len(a[0, :])), a[0, :])

Vì có một số căn chỉnh sơ bộ giữa các từ đầu vào và đầu ra, bạn mong đợi sự chú ý sẽ được tập trung gần đường chéo:

In [ ]:
plt.imshow(np.array(a), vmin=0.0)

Dưới đây là một số mã để tạo ra một đồ thị chú ý tốt hơn:

In [ ]:
#@title Đồ thị chú ý có nhãn
def plot_attention(attention, sentence, predicted_sentence):
  sentence = tf_lower_and_split_punct(sentence).numpy().decode().split()
  predicted_sentence = predicted_sentence.numpy().decode().split() + ['[END]']
  fig = plt.figure(figsize=(10, 10))
  ax = fig.add_subplot(1, 1, 1)

  attention = attention[:len(predicted_sentence), :len(sentence)]

  ax.matshow(attention, cmap='viridis', vmin=0.0)

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  ax.set_xlabel('Input text')
  ax.set_ylabel('Output text')
  plt.suptitle('Attention weights')

In [ ]:
i=0
plot_attention(result['attention'][i], input_text[i], result['text'][i])

Dịch thêm một vài câu và vẽ sơ đồ:

In [ ]:
%%time
three_input_text = tf.constant([
    # This is my life.
    'Esta es mi vida.',
    # Are they still home?
    '¿Todavía están en casa?',
    # Try to find out.'
    'Tratar de descubrir.',
])

result = translator.tf_translate(three_input_text)

for tr in result['text']:
  print(tr.numpy().decode())

print()

In [ ]:
result['text']

In [ ]:
i = 0
plot_attention(result['attention'][i], three_input_text[i], result['text'][i])

In [ ]:
i = 1
plot_attention(result['attention'][i], three_input_text[i], result['text'][i])

In [ ]:
i = 2
plot_attention(result['attention'][i], three_input_text[i], result['text'][i])

Các câu ngắn thường hoạt động tốt, nhưng nếu đầu vào quá dài, mô hình sẽ mất tập trung theo đúng nghĩa đen và ngừng cung cấp các dự đoán hợp lý. Có hai lý do chính cho việc này:

1. Mô hình đã được huấn luyện với việc teacher-forcing cung cấp đúng token ở mỗi bước, bất kể dự đoán của mô hình. Mô hình có thể được thực hiện mạnh mẽ hơn nếu đôi khi nó được cung cấp các dự đoán của chính nó.

2. Mô hình chỉ có quyền truy cập vào đầu ra trước đó của nó thông qua trạng thái RNN. Nếu trạng thái RNN bị hỏng, không có cách nào để mô hình phục hồi. [Transformers](https://www.tensorflow.org/text/tutorials/transformer) giải quyết điều này bằng cách sử dụng tự chú ý trong bộ mã hóa và giải mã.

In [ ]:
long_input_text = tf.constant([inp[-1]])

import textwrap
print('Expected output:\n', '\n'.join(textwrap.wrap(targ[-1])))

In [ ]:
result = translator.tf_translate(long_input_text)

i = 0
plot_attention(result['attention'][i], long_input_text[i], result['text'][i])
_ = plt.suptitle('This never works')

## Xuất

Một khi bạn có một mô hình đang hài lòng, bạn có thể muốn xuất nó như là một `tf.saved_model` để sử dụng bên ngoài chương trình python này đã tạo ra nó.

Từ mô hình này là một lớp con của `tf.Module` (thông qua `keras.Model`), và tất cả các hàm phục vụ xuất được biên dịch trong một `tf.function` mô hình nên xuất sạch với `tf.saved_model.save`:

Bây giờ mà hàm đã được bắt nguồn từ nó có thể được xuất sử dụng `saved_model.save`:

In [ ]:
tf.saved_model.save(translator, 'translator',
                    signatures={'serving_default': translator.tf_translate})

In [ ]:
reloaded = tf.saved_model.load('translator')
result = reloaded.tf_translate(three_input_text)

In [ ]:
%%time
result = reloaded.tf_translate(three_input_text)

for tr in result['text']:
  print(tr.numpy().decode())

print()